In [1]:
import pandas as pd
import json
import os
from PIL import Image
import numpy as np

In [2]:
def coco_to_yolo_coord(v):
    width,height,xmin,xmax,ymin,ymax = v[0],v[1],v[2],v[3],v[4],v[5]
    x = (xmax-xmin)/2
    x = x/width
    y= (ymax-ymin)/2
    y = y/height
    w = xmax-xmin
    w = w/width
    h = ymax-ymin
    h = h/height
    return x,y,w,h

In [3]:
def coco_class_to_yolo(class_json,w,h):
    class_id = class_json['classId'] -1
    points = list(class_json['points'].values())
    x,y,w,h = coco_to_yolo_coord([w,h]+points)
    yolo_string = "{} {} {} {} {}".format(str(class_id),x,y,w,h)
    return yolo_string

In [4]:
files = os.listdir('Annotations')
#number of files
num_files = len(files)

In [5]:
classes_index = files.index('classes.json')

In [6]:
#superannotate tool stores classes.json in annotations folder
classes = files[classes_index]
files.remove('classes.json')
print(len(files))

112


In [7]:
#Convert coco json of all images to coco
for i in files[:]:
    image_name = i[:-9]
    with open('Annotations/'+image_name+'.png.json') as f:
        data = json.load(f)
    img = np.asarray(Image.open('Yolo_data/Images/'+image_name+'.jpg'))
    h,w = img.shape[0],img.shape[1]
    s=""
    for j in data['instances']:
        s += coco_class_to_yolo(j,w,h)+"\n"
    with open('Yolo_data/Images/'+image_name+'.txt','w') as t:
        t.write(s)
        t.close()


In [8]:
#generate train.txt
train = files[:int(0.8*num_files)]
with open('Yolo_data/train.txt','w') as tr:
    for i in train:
        tr.write('Images/'+i[:-9]+".jpg\n")
    tr.close()
#generate test.txt
test = files[int(0.8*num_files):]
with open('Yolo_data/test.txt','w') as te:
    for i in test:
        te.write('Images/'+i[:-9]+".jpg\n")
    te.close()

In [9]:
with open('Annotations/classes.json','r') as c:
    classes = json.load(c)
with open('Yolo_data/obj.names','w') as on:
    for i in classes:
        on.write(i['name']+'\n')
    on.close()